In [65]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from time import sleep,time
from random import randint
from IPython.core.display import clear_output

In [69]:
import json


In [71]:
response=get('http://cadd.pharmacy.nankai.edu.cn/yatcm_api/prescriptions/?&include[]=herbs.id&exclude[]=herbs.*&page=1&per_page=100&sort[]=english_name')

In [72]:
data=BeautifulSoup(response.text,'html.parser')

In [74]:
data

<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr/><center>none</center>
</body>
</html>

In [73]:
newDictionary=json.loads(str(data))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
newDictionary['prescriptions'][0]['english_name']

In [47]:
chinese_name=[]
english_name=[]
phonetic_name=[]
modern_app=[]
trad_app=[]

# Preparing the monitoring of the loop
start_time = time()
requests = 0


In [42]:
page=2
page_o=100
response=get('http://cadd.pharmacy.nankai.edu.cn/yatcm_api/prescriptions/?&include[]=herbs.id&exclude[]=herbs.*&page='+str(page) +'&per_page='+str(page_o)+'&sort[]=english_name')
data=BeautifulSoup(response.text,'html.parser')
newDictionary=json.loads(str(data)) 


In [45]:
newDictionary['prescriptions'][0]['english_name']

'sangxing decoction'

In [49]:


chinese_name=[]
english_name=[]
phonetic_name=[]
modern_app=[]
trad_app=[]

# Preparing the monitoring of the loop
start_time = time()
requests = 0
page=1
while page<=100:
    
    response=get('http://cadd.pharmacy.nankai.edu.cn/yatcm_api/prescriptions/?&include[]=herbs.id&exclude[]=herbs.*&page='+str(page) +'&per_page=100&sort[]=english_name')
    data=BeautifulSoup(response.text,'html.parser')
    newDictionary=json.loads(str(data)) 
     # Pause the loop
    sleep(randint(8,15))

    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    for i in range(0,99):     
        english_name.append(newDictionary['prescriptions'][i]['english_name']) 
        chinese_name.append(newDictionary['prescriptions'][i]['chinese_name'])
        phonetic_name.append(newDictionary['prescriptions'][i]['phonetic_name'])
        modern_app.append(newDictionary['prescriptions'][i]['modern_application'])
        trad_app.append(newDictionary['prescriptions'][i]['traditional_application'])       
    page+=1

Request:5; Frequency: 0.05669135310997467 requests/s


In [ ]:
imdb_data=pd.DataFrame(
    {
        "english_name":english_name,
        "chinese_name":chinese_name,
        "phonetic_name":phonetic_name,
        "modern_application":modern_app,
        "traditional_application":trad_app
    }
)

In [64]:
english_name[151]

'zhengan xifeng decoction'

In [138]:
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

In [144]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url +
        '&sort=num_votes,desc&page=' + page)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.0625088957846182 requests/s


In [145]:
imdb_data=pd.DataFrame(
    {
        "movie":names,
        "year":years,
        "imdb_ratings":imdb_ratings,
        "metascore":metascores,
        "votes":votes
    }
)

In [154]:
imdb_data.loc[:,'year']=imdb_data['year'].str[-5:-1].astype(int)

In [155]:
imdb_data.to_csv("E://DATA Science//Datasets//IMDB//imdb_rating.csv")